# Pasos:

1. Particionar datos de Entrenamiento (80%) y Pruebas(20%) 
2. Entrenar Modelos a evaluar:
    - XGBoost
    - SVM
    - Ensamble Modelo (RandomForest o Arbol de decisión)
3. Evaluar los hiperparametros (RandomSearchCV)
4. Entrenar modelos con mejores hiperparámetros encontrados en 3.
5. Evaluar modelos con data de pruebas (Accuracy)

## 1) Lectura de datos - Distribución de conjunto de datos

La distribución del conjunto de datos será de la siguiente manera: 

- Entrenamiento (80%)
- Pruebas (20%)

--- Ya fue aplicado en 2. Pre-procesamiento de datos

In [27]:
import pandas as pd
import numpy as np
import os
import csv
import matplotlib.pyplot as plt

from sklearn import preprocessing, neighbors
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from pprint import pprint

from sklearn.model_selection import RandomizedSearchCV

In [2]:
#Lectura del dataset

X_train =  pd.read_csv(
    'data/03_entrada_modelo/X_MI_entrenamiento_caracterizado.csv', 
    header=0, 
    sep=","
)
X_test =  pd.read_csv(
    'data/03_entrada_modelo/X_MI_pruebas_caracterizado.csv', 
    header=0, 
    sep=","
)
y_train =  pd.read_csv(
    'data/03_entrada_modelo/y_MI_entrenamiento_caracterizado.csv', 
    header=0, 
    sep=","
)
y_test =  pd.read_csv(
    'data/03_entrada_modelo/y_MI_pruebas_caracterizado.csv', 
    header=0, 
    sep=","
)

## 2) Entrenar Modelos a evaluar

In [39]:
## Resultados con la métrica de exactitud (Accuracy) de cada uno de los modelos.
resultados_modelo_acc = pd.DataFrame(columns=["Modelo","Accuracy"])

### 2.1) Random Forest

Entrenamos el modelo de Random Forest de tipo clasificación.

In [3]:
rf = RandomForestClassifier(oob_score = True)
rf.fit(X_train, y_train)

print('Exactitud del modelo inicial en entrenamiento:', rf.score(X_train, y_train))
print('Exactitud del modelo inicial en entrenamiento (Out of Bag):', rf.oob_score_)
print('Exactitud del modelo inicial en validación:', rf.score(X_test, y_test))

<ipython-input-3-ddb954a81c03>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


Exactitud del modelo inicial en entrenamiento: 1.0
Exactitud del modelo inicial en entrenamiento (Out of Bag): 0.825735294117647
Exactitud del modelo inicial en validación: 0.8088235294117647


#### Búsqueda aleatoria de mejores hiperparámetros

In [4]:
# Definición de Grilla
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 10, 11, 12]  # 'auto' equivale a 'sqrt'; None equivale a todas
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)] + [None]
min_samples_leaf = [1, 2, 4]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_leaf': min_samples_leaf}

print('Los valores a probar en la búsqueda aleatoria son:')
pprint(random_grid)

print()
print('Si se probara todas las combinaciones se requeriría entrenar', 
      len(random_grid['n_estimators']) *
      len(random_grid['max_features']) *
      len(random_grid['max_depth']) *
      len(random_grid['min_samples_leaf']),
      'modelos'
      )

rf = RandomForestClassifier(oob_score=True)
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               cv = 3,          # Validación cruzada 3-fold
                               verbose=2, 
                               random_state=0, 
                               n_jobs = -1      # Paralelizar en todos los cores disponibles
                               )
rf_random.fit(X_train, y_train)

Los valores a probar en la búsqueda aleatoria son:
{'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 10, 11, 12],
 'min_samples_leaf': [1, 2, 4],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

Si se probara todas las combinaciones se requeriría entrenar 1440 modelos


In [6]:
## Tomamos el mejor estimador encontrado en la búsqueda aleatoria por grilla.
rf_random_best = rf_random.best_estimator_

print('Los hiperparámetros del mejor modelo son:')
pprint(rf_random.best_params_)
print()

print('Exactitud luego de búsqueda aleatoria en entrenamiento:', rf_random_best.score(X_train, y_train))
print('Exactitud luego de búsqueda aleatoria en entrenamiento (Out of Bag):', rf_random_best.oob_score_)
print('Exactitud luego de búsqueda aleatoria en validación:', rf_random_best.score(X_test, y_test))

Los hiperparámetros del mejor modelo son:
{'max_depth': None,
 'max_features': 11,
 'min_samples_leaf': 1,
 'n_estimators': 800}

Exactitud luego de búsqueda aleatoria en entrenamiento: 1.0
Exactitud luego de búsqueda aleatoria en entrenamiento (Out of Bag): 0.8529411764705882
Exactitud luego de búsqueda aleatoria en validación: 0.8264705882352941


### 2.2) Ensamble - XGBoost

In [7]:
from xgboost import XGBRegressor
from pprint import pprint
from sklearn.model_selection import GridSearchCV
import time

## XGBoostRegressor
modelXGBR = XGBRegressor()


## Creamos la Grilla
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 5)]
learning_rate = [0.01,0.1]
max_depth = [i for i in range(2,8,2)]
start = time.time()

random_grid = {'max_depth': max_depth,
               'n_estimators': n_estimators,
               'colsample_bytree': [0.2, 0.6, 0.8],
               'min_child_weight': [3, 5, 7],
               'gamma': [0.3, 0.5, 0.7],
               'subsample': [0.4, 0.6, 0.8, 1],
               'learning_rate': learning_rate}

#### Búsqueda aleatoria de mejores hiperparámetros

In [8]:
xgb_random = RandomizedSearchCV(estimator = modelXGBR, 
                               param_distributions = random_grid, 
                               cv = 3,          # Validación cruzada 3-fold
                               verbose=2, 
                               random_state=0, 
                               n_jobs = -1      # Paralelizar en todos los cores disponibles
                               )
xgb_random.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          enable_categorical=False, gamma=None,
                                          gpu_id=None, importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=...
                                          reg_alpha=None, reg_lambda=None,
                                          scale_pos_weight=None, subsample=None,
                                          tree_method=None,
                                          validate_parameters=None,
                                          verbosity=None),
                   n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.2, 0.6, 0.8],
                                        'gamma': [0.3, 0.5, 0.7],
                                        'learning_rate': [0.01, 0.1],
                                        'max_depth': [2, 4, 6],
                                        'min_child_weight': [3, 5, 7],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500],
                                        'subsample': [0.4, 0.6, 0.8, 1]},
                   random_state=0, verbose=2)

In [9]:
## Tomamos el mejor estimador encontrado en la búsqueda aleatoria por grilla.
xgb_best_model = xgb_random.best_estimator_

print('Los hiperparámetros del mejor modelo son:')
pprint(xgb_random.best_params_)
print()


score_XGBR_train = xgb_best_model.score(X_train, y_train)
score_XGBR_val = xgb_best_model.score(X_test, y_test)
print("*"*100)
print(f"Exactitud luego de búsqueda en grilla en entrenamiento: {score_XGBR_train*100:.3f}%")
print(f"Exactitud luego de búsqueda en grilla en validación: {score_XGBR_val*100:.3f}%")
print("*"*100)
end = time.time()
print(f"{end-start} segundos")

Los hiperparámetros del mejor modelo son:
{'colsample_bytree': 0.6,
 'gamma': 0.5,
 'learning_rate': 0.1,
 'max_depth': 2,
 'min_child_weight': 5,
 'n_estimators': 400,
 'subsample': 0.6}

****************************************************************************************************
Exactitud luego de búsqueda en grilla en entrenamiento: 99.987%
Exactitud luego de búsqueda en grilla en validación: 99.711%
****************************************************************************************************
12.87302279472351 segundos


### 2.3) Ensamble - SVM

In [10]:
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier

modelSVM = svm.SVC()

# Entrenamiento del modelo base - SVM
modelSVM.fit(X_train, y_train)
score_SVM_train = modelSVM.score(X_train, y_train)
score_SVM_test = modelSVM.score(X_test, y_test)


print(f"R^2: {score_SVM_train}")
print(f"R^2: {score_SVM_test}")

D:\Users\coliverac\Anaconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


R^2: 0.861764705882353
R^2: 0.4823529411764706


#### Búsqueda aleatoria de mejores hiperparámetros

In [36]:
# Definición de grilla
random_grid = {'kernel': ['rbf'],
               'C': np.logspace(-4,4,9), # [0.0001, 0.001, ..., 10000]
               'gamma': np.logspace(-4,4,9)  # [0.0001, 0.001, ..., 10000]
              }

modelSVM_random = RandomizedSearchCV(estimator = modelSVM, 
                               param_distributions = random_grid, 
                               cv = 12,          # Validación cruzada 3-fold
                               verbose=2, 
                               random_state=0, 
                               n_jobs = -1      # Paralelizar en todos los cores disponibles
                               )
modelSVM_random.fit(X_train, y_train)

## Tomamos el mejor estimador encontrado en la búsqueda aleatoria por grilla.
modelSVM_best_model = modelSVM_random.best_estimator_

print('Los hiperparámetros del mejor modelo son:')
pprint(modelSVM_random.best_params_)
print()


score_SVM_train = modelSVM_best_model.score(X_train, y_train)
score_SVM_test = modelSVM_best_model.score(X_test, y_test)
print("*"*100)
print(f"Exactitud luego de búsqueda en grilla en entrenamiento: {score_SVM_train:.3f}")
print(f"Exactitud luego de búsqueda en grilla en validación: {score_SVM_test:.3f}")

end = time.time()
print(end-start)

Fitting 12 folds for each of 10 candidates, totalling 120 fits


D:\Users\coliverac\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=12.
  warnings.warn(
D:\Users\coliverac\Anaconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Los hiperparámetros del mejor modelo son:
{'C': 100.0, 'gamma': 0.001, 'kernel': 'rbf'}

****************************************************************************************************
Exactitud luego de búsqueda en grilla en entrenamiento: 0.854
Exactitud luego de búsqueda en grilla en validación: 0.821
1222.3481879234314
